In [26]:
#Import Libraries

from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [27]:
!pip install wandb -qqq
import wandb

In [28]:
# Log in to your W&B account
wandb.login()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [29]:
args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=1000
args['epochs']=10  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=True

In [31]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw



In [37]:

class Net(nn.Module):
  #This defines the structure of the NN.
  def __init__(self):
      super(Net, self).__init__()
      self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
      self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
      self.conv2_drop = nn.Dropout2d()  #Dropout
      self.fc1 = nn.Linear(320, 50)
      self.fc2 = nn.Linear(50, 10)

  def forward(self, x):
      #Convolutional Layer/Pooling Layer/Activation
      x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
      #Convolutional Layer/Dropout/Pooling Layer/Activation
      x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
      x = x.view(-1, 320)
      #Fully Connected Layer/Activation
      x = F.relu(self.fc1(x))
      x = F.dropout(x, training=self.training)
      #Fully Connected Layer/Activation
      x = self.fc2(x)
      #Softmax gets probabilities. 
      return F.log_softmax(x, dim=1)
            

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False)
        pred = output.data.max(1, keepdim=True)[1] 
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [38]:
wandb.init(
    project="mnist example",
    config=args)
config = wandb.config

model = Net()
wandb.watch(model)
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss))
        wandb.log({"loss": loss})
    test()

loss,2.1826


loss,█▇▇▇▇▇▆▇▇▆▆▆▆▆▆▆▅▆▆▅▅▅▅▅▅▄▄▃▄▄▄▄▃▄▂▃▂▃▂▁


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.321507
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.306127
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.297818
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.282647
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.278510
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.264506


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 2.2304, Accuracy: 3557/10000 (36%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.244985
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.229261
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.197716
Train Epoch: 2 [30000/60000 (50%)]	Loss: 2.177804
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.125686
Train Epoch: 2 [50000/60000 (83%)]	Loss: 2.077452

Test set: Average loss: 1.8987, Accuracy: 5928/10000 (59%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.000676
Train Epoch: 3 [10000/60000 (17%)]	Loss: 1.912232
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.804156
Train Epoch: 3 [30000/60000 (50%)]	Loss: 1.768737
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.625240
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.565830

Test set: Average loss: 1.0704, Accuracy: 8223/10000 (82%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 1.381911
Train Epoch: 4 [10000/60000 (17%)]	Loss: 1.350726
Train Epoch: 4 [20000/60000 (33%)]	Loss: 1.282245
Train Epoch: 4 [30000/60000 (50%)]	Loss: 1.222708
Train Epoch: 4 [40000/60000 (67%